# IOC Case Study #11: `mshta.exe` Execution (Unexpected HTA Launch)

---

## **Technical Definition**

`mshta.exe` is a legitimate Windows utility located in the System32 directory that executes HTML Application (HTA) files. HTA (HTML Application) files are used to run HTML-based applications that can include embedded scripting languages such as VBScript or JavaScript, allowing developers or administrators to create interactive, GUI-based tools that run with the full privileges of the Windows host system. `mshta.exe` is often abused by attackers because it is a signed, trusted Microsoft binary that can be used to execute scripts locally or from remote URLs without dropping a traditional executable file.

---

## **Source of IOC (Telemetry Origin)**

This IOC was detected through:

- **Windows Security Event Logs**, showing a suspicious process creation involving `mshta.exe`.
- **EDR telemetry**, which flagged abnormal usage of `mshta.exe` as a scripting engine.
- **SIEM rules** may also generate alerts when common scripting binaries are invoked with unusual command-line patterns.

---

## **Triage Framework Declaration**

**Host-Based Triage Protocol**

### **Standard Investigative Toolkit (in strict sequence)**

1. **Windows Event Logs**  
   - **Event ID 4688 – Process Creation**: Indicates a new process was launched, including the full command line. Use this to identify who or what invoked `mshta.exe`.
   - **Event ID 7045 – New Service Installation**: May appear *instead of* 4688 if `mshta.exe` was registered as a persistent service.
   - Event IDs **4624 (Logon Success)** or **4625 (Logon Failure)** may correlate to the user or attacker account involved in the invocation.

2. **EDR Telemetry (if available)**  
   - **Process Lineage**: Displays parent-child relationships (e.g., `explorer.exe` → `mshta.exe`), revealing if the launch was user-driven or script-triggered.
   - **Command-Line Parameters**: Shows arguments passed to `mshta.exe`, such as JavaScript, VBScript, or URLs. This helps uncover payload locations or obfuscation techniques.
   - **Behavioral Flags**: Identifies abnormal use patterns, such as obfuscated scripts or HTA payloads from untrusted web domains.

3. **File System and Registry Inspection**  
   - Check for **`.hta` files** stored locally. These may be dropped as temporary payloads in directories like `%TEMP%` or `%APPDATA%`.
   - Inspect **registry autorun keys** (e.g., `HKCU\Software\Microsoft\Windows\CurrentVersion\Run`) for persistence referencing `mshta.exe`.
   - Investigate **Scheduled Tasks** using `mshta.exe`. Use Task Scheduler GUI or `schtasks /query` to list and examine scheduled jobs.
   - Validate the integrity of `C:\Windows\System32\mshta.exe` by comparing its hash with a known good version to rule out tampering.

4. **Volatile Memory Capture**  
   - Use tools like **Rekall** or **Volatility** to locate `mshta.exe` in memory.
   - Extract in-memory scripts or loaded modules and analyze for injected content or outbound connection attempts.
   - Confirm any open network sockets established by the `mshta.exe` process.

### **Supplemental Tools (Only if Core Toolkit Is Exhausted)**

- **Process Explorer**: Visual tool showing live process trees, memory usage, and execution details. Useful for identifying real-time child processes spawned by `mshta.exe`.
- **Sysmon (Event ID 1 – Process Creation)**: Provides richer visibility into process launches, including extended command-line data and hash values.
- **Autoruns**: Enumerates all auto-starting registry keys and startup folders. Helps detect persistent use of `mshta.exe` across multiple launch points.

---

## **Host OS Layer (Six-Layer Model Mapping)**

- **Layer 1 – Process Execution Environment**: `mshta.exe` is directly executed to launch script logic.
- **Layer 2 – Startup and Persistence Infrastructure**: Registry autoruns or scheduled tasks may invoke `mshta.exe` at startup or login.
- **Layer 6 – Network Communication Context**: `mshta.exe` may reach out to attacker-controlled web domains to fetch HTA content.

---

## **Cross-Layer Interaction Pivots**

- **Layer 2 → Layer 1**: Registry-based autorun or scheduled task triggers `mshta.exe`.
- **Layer 1 → Layer 6**: Execution of `mshta.exe` initiates outbound HTTP/HTTPS requests.
- **Layer 1 → Layer 5**: EDR or antivirus systems detect behavior anomalies associated with `mshta.exe`.

---

## **OSI Layer Relevance**

- **Layer 7 – Application Layer**: `mshta.exe` retrieves a remote HTA payload via HTTPS.
- **Layer 4 – Transport Layer**: Connections occur over TCP, typically port 80 (HTTP) or 443 (HTTPS).
- **Layer 3 – Network Layer**: DNS queries and IP routing direct the request to the attacker’s infrastructure.

---

## **Attacker Behavior Interpretation**

- `mshta.exe` is a **living-off-the-land binary (LOLBin)** commonly abused to avoid detection.
- The attacker avoids dropping executable payloads to disk, reducing the chance of detection by file-based antivirus. Instead, they use scripting logic to operate in-memory or stream code over the network.
- **Chaining trusted binaries** refers to a technique where the attacker uses one or more built-in Windows tools (like `mshta.exe`, `rundll32.exe`, or `powershell.exe`) in sequence to download, execute, or escalate without introducing foreign executables.
- This method is common in **phishing-based delivery**, where a user is tricked into clicking a link or opening a document that invisibly calls `mshta.exe`.
- **Stage-one execution chains** are initial foothold payloads (e.g., droppers, downloaders, scripts) that lead to more powerful malware in later stages.
- `mshta.exe` can either execute **inline script content** or fetch full `.hta` files from the internet, allowing attackers to modularize and update payloads dynamically.

---

## **Defender Action Summary**

- Search **Event ID 4688 (Process Creation)** logs for `mshta.exe`, and capture full command-line details.
- Use EDR tools to trace **process lineage**, highlight unusual parent processes, or detect long, encoded arguments.
- Examine **registry autoruns** and **scheduled tasks** that might reference `mshta.exe` for persistence.
- Capture and detonate any referenced `.hta` files in a sandboxed environment to observe their behavior.
- Examine **network telemetry** to identify:
  - Outbound connections from `mshta.exe` processes
  - Communication with suspicious domains that have no browser-based traffic
- If not essential for business use, restrict or disable `mshta.exe` via AppLocker or Windows Defender Application Control (WDAC).
- Implement SIEM detection rules for:
  - `mshta.exe` calling external domains
  - `mshta.exe` with unusually long or encoded command lines

---

## **Attacker Strategy Notes**

- Attackers leverage **dual-use system tools**—legitimate Windows utilities like `mshta.exe` that serve both administrative and malicious purposes—because they are pre-installed and signed.
- Payloads delivered through `mshta.exe` are often **obfuscated** and hosted on **throwaway infrastructure**, meaning domains or IPs created specifically for one short-term campaign and discarded afterward.
- The attacker seeks a **minimal file system footprint**, launching scripts or malware directly in memory to evade antivirus and limit forensic artifacts.
- This approach is often part of an **automated, modular toolkit**, where tools can be swapped in and out as needed—making `mshta.exe` a convenient **plug-and-play launcher**.
- This behavior is used by **advanced persistent threat (APT) actors** as well as **commodity malware campaigns**, making it a highly relevant tactic for defenders to detect.

---
